In [1]:
import casadi
import numpy as np

# 例１
- https://github.com/gomi-kuzu/practice_optimize/blob/master/transport/practice_solver.ipynb
- https://qiita.com/gomi-kuzu/items/9a4c5743737fa0731c81

In [2]:
#問題設定
supry = np.array([10,5,2]) # 倉庫の方の供給
demand = np.array([7,10]) # 客の方の需要
C = np.array([[1.,2.,3.], [4.,5.,6.]]) 

In [3]:
X = casadi.SX.sym("x", (2,3)) #最適化の対象となる変数を定義

In [4]:
C = casadi.SX(C) #コスト

In [5]:
f = casadi.sum1(casadi.sum2(X*C)) #目的関数
#casadi.sum1が列ごとのsum, sum2が行ごとのsum

In [6]:
#倉庫と客の制約をひとつのベクトルにconcatする
d =casadi.vertcat(casadi.sum1(X).T, casadi.sum2(X))
b = casadi.vertcat(supry, demand)
g = d-b

In [7]:
nlp = {} #変数、目的関数、制約は辞書型で渡す
nlp["x"] = X.reshape((-1, 1)) #変数はベクトルで渡さなくてはいけない
nlp["f"] = f 
nlp["g"] = g
# reshapeの展開の仕方がnumpyのソレと微妙に違うので注意

In [8]:
F = casadi.nlpsol("F", "ipopt", nlp) # ファンクションとして登録

In [9]:
sol = F(ubg=0,lbg=0, lbx=0)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

In [10]:
sol

{'f': DM(56),
 'g': DM([0, 0, -2.22045e-16, 0, 0]),
 'lam_g': DM([-37196, -37197, -37198, 37195, 37192]),
 'lam_p': DM([]),
 'lam_x': DM([-6.51178e-10, -4.07356e-10, -1.13888e-09, -8.95062e-10, -2.63372e-09, -2.3899e-09]),
 'x': DM([4.37317, 5.62683, 2.03609, 2.96391, 0.590738, 1.40926])}

In [11]:
casadi.SX(sol["x"]).reshape((2,3)) #元の行列の形に直す

SX(
[[4.37317, 2.03609, 0.590738], 
 [5.62683, 2.96391, 1.40926]])

## 線形ソルバーで解いてみる

In [12]:
lp = {} #変数、目的関数、制約は辞書型で渡す
lp["x"] = X.reshape((-1, 1)) #変数はベクトルで渡さなくてはいけない
lp["f"] = f 
lp["g"] = g

tes = casadi.qpsol("F", "cbc", lp) # ファンクションとして登録

In [13]:
sol_tes = tes(ubg=0,lbg=0, lbx=0)

Welcome to the CBC MILP Solver 
Version: 2.9.6 
Build Date: Aug  7 2018 

command line - CbcInterface -solve -quit (default strategy 1)
Presolve 0 (-5) rows, 0 (-6) columns and 0 (-12) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 56
After Postsolve, objective 56, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 56 - 0 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [14]:
sol_tes

{'f': DM(56),
 'g': DM([0, 0, 0, 0, 0]),
 'lam_g': DM([-4, -5, -6, 3, -0]),
 'lam_p': DM([]),
 'lam_x': DM([-0, -0, -0, -0, -0, -0]),
 'x': DM([7, 3, 0, 5, 0, 2])}

# 例2
- https://www.msi.co.jp/nuopt/docs/v19/examples/html/02-02-00.html

In [15]:
supry = np.array([250,450]) # 倉庫の方の供給
demand = np.array([200,200,200]) # 客の方の需要
C = np.array([[3.4,2.2], [2.9, 3.4], [2.4, 2.5]]) 

In [16]:
X = casadi.SX.sym("x", (3,2))
C = casadi.SX(C)

In [17]:
f = casadi.sum1(casadi.sum2(X*C))

In [20]:
d =casadi.vertcat(casadi.sum1(X).T, casadi.sum2(X), -1*casadi.sum2(X)) #等式制約のために-1かけて増やす
b = casadi.vertcat(supry, demand, -1*demand)

In [22]:
g = d-b

In [24]:
nlp = {} #変数、目的関数、制約は辞書型で渡す
nlp["x"] = X.reshape((-1, 1))
nlp["f"] = f 
nlp["g"] = g

In [25]:
F = casadi.nlpsol("F", "ipopt", nlp) 
sol = F(ubg=0, lbx=0) #等式制約を不等式制約に変換してる

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       18
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        8
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        8

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [26]:
sol["x"]

DM([0, 200, 50, 200, 0, 150])

In [27]:
sol

{'f': DM(1515),
 'g': DM([-1.5059e-08, -100, -9.02784e-09, -9.22273e-09, -9.06721e-09, 9.02784e-09, 9.22273e-09, 9.06721e-09]),
 'lam_g': DM([0.1, 2.5034e-11, 0.143652, 0.133999, 0.133992, 2.34365, 3.134, 2.63399]),
 'lam_p': DM([]),
 'lam_x': DM([-1.3, -1.25295e-11, -5.01181e-11, -1.25295e-11, -0.4, -1.6706e-11]),
 'x': DM([0, 200, 50, 200, 0, 150])}

In [28]:
casadi.SX(sol["x"]).reshape((3,2))

SX(@1=0, 
[[@1, 200], 
 [200, @1], 
 [50, 150]])

## 同じくCBCで解く

In [29]:
lp = {}
lp["x"] = X.reshape((-1, 1)) # 複数変数がある場合はcasadi.vertcat()関数を使ってひとつの構造体にする
lp["f"] = f 
lp["g"] = g

In [30]:
F = casadi.qpsol("F", "cbc", lp) # ファンクションとして登録
sol = F(ubg=0, lbx=0) #等式制約を不等式制約に変換してる

Welcome to the CBC MILP Solver 
Version: 2.9.6 
Build Date: Aug  7 2018 

command line - CbcInterface -solve -quit (default strategy 1)
Presolve 8 (0) rows, 6 (0) columns and 18 (0) elements
0  Obj 0 Primal inf 600 (3)
4  Obj 1515
Optimal - objective value 1515
Optimal objective 1515 - 4 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [31]:
print("目的関数：{}".format(sol["f"]))
print("変数：{}".format(casadi.SX(sol["x"]).reshape((2,3))))

目的関数：1515
変数：@1=0, @2=200, 
[[@1, 50, @1], 
 [@2, @2, 150]]


- 使えるソルバーっっぽいもの一覧
    - `l /home/inoma/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/casadi/`